## Import

In [19]:
import os
import time
import glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# sklearn
from sklearn.metrics import log_loss, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold,StratifiedShuffleSplit, train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import random

# tensorflow
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

## Data Load / Preprocessing

In [20]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

train = train.drop(columns=['id', 'father', 'mother', 'gender'])
test = test.drop(columns=['id', 'father', 'mother', 'gender'])

In [23]:
TARLE = LabelEncoder()
SNPLE = LabelEncoder()
SNPCOL = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]

snp_data = []
for col in SNPCOL:
    snp_data += list(train[col].values)
    
train.iloc[:,-1] = TARLE.fit_transform(train.iloc[:,-1])
SNPLE.fit(snp_data)

for col in train.columns:
    if col in SNPCOL:
        train[col] = SNPLE.transform(train[col])
        test[col] = SNPLE.transform(test[col])

## StraitfiedSS

In [24]:
# provides train/test indices to split data in train/test sets.
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)



for train_idx, test_idx in split.split(train, train['trait']):

    x_train = train.iloc[:,:-1].loc[train_idx]

    x_test = train.iloc[:,:-1].loc[test_idx]

    y_train = train.iloc[:,-1].loc[train_idx]

    y_test = train.iloc[:,-1].loc[test_idx]
    
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    print('오차행렬 \n', confusion_matrix(y_test, pred))
    print('정확도 :', accuracy_score(y_test, pred))
    print('정밀도 : ',precision_score(y_test, pred, average='macro'))
    print('재현율 :', recall_score(y_test, pred, average='macro'))
    print('f1 score :', f1_score(y_test, pred, average='macro'))

In [26]:
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import SuccessiveHalvingPruner
import lightgbm as lgb
from lightgbm import LGBMClassifier
def objective(trial):
    params = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 3,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    score = []
    clf = LGBMClassifier(**params)
    clf.fit(x_train, y_train)
    
    pred = clf.predict(x_test)
    score = accuracy_score(y_test, pred)
    return score

# Hyperparameter Tuning
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=0), pruner=SuccessiveHalvingPruner())
study.optimize(objective, n_trials=2000)


[I 2022-12-28 14:24:42,475] A new study created in memory with name: no-name-debee562-6d33-4d54-8ab9-125d84a4c1e0
[I 2022-12-28 14:24:42,597] Trial 0 finished with value: 0.8301886792452831 and parameters: {'lambda_l1': 0.0008696040132105586, 'lambda_l2': 0.02733406969031059, 'num_leaves': 155, 'feature_fraction': 0.7269299097981381, 'bagging_fraction': 0.6541928796033428, 'bagging_freq': 5, 'min_child_samples': 47}. Best is trial 0 with value: 0.8301886792452831.
[I 2022-12-28 14:24:42,615] Trial 1 finished with value: 0.39622641509433965 and parameters: {'lambda_l1': 1.0615904599004025, 'lambda_l2': 4.7093920302768835, 'num_leaves': 99, 'feature_fraction': 0.8750350228495988, 'bagging_fraction': 0.7173369518517427, 'bagging_freq': 4, 'min_child_samples': 93}. Best is trial 0 with value: 0.8301886792452831.
[I 2022-12-28 14:24:42,633] Trial 2 finished with value: 0.39622641509433965 and parameters: {'lambda_l1': 4.3583742877412676e-08, 'lambda_l2': 6.083642669826052e-08, 'num_leaves':

[LightGBM] [Warning] feature_fraction is set=0.7269299097981381, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7269299097981381
[LightGBM] [Warning] lambda_l1 is set=0.0008696040132105586, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0008696040132105586
[LightGBM] [Warning] bagging_fraction is set=0.6541928796033428, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6541928796033428
[LightGBM] [Warning] lambda_l2 is set=0.02733406969031059, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.02733406969031059
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8750350228495988, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8750350228495988
[LightGBM] [Warning] lambda_l1 is set=1.0615904599004025, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.0615904599004025
[LightGBM] [Warning] bagging_

[I 2022-12-28 14:24:42,732] Trial 4 finished with value: 0.9245283018867925 and parameters: {'lambda_l1': 0.000497318821411805, 'lambda_l2': 5.3945777886906086e-05, 'num_leaves': 69, 'feature_fraction': 0.86454021366053, 'bagging_fraction': 0.6736901993299291, 'bagging_freq': 4, 'min_child_samples': 6}. Best is trial 4 with value: 0.9245283018867925.
[I 2022-12-28 14:24:42,779] Trial 5 finished with value: 0.8113207547169812 and parameters: {'lambda_l1': 0.0036200908827975302, 'lambda_l2': 0.003227464726605418, 'num_leaves': 159, 'feature_fraction': 0.9662488471087745, 'bagging_fraction': 0.80909217946209, 'bagging_freq': 3, 'min_child_samples': 46}. Best is trial 4 with value: 0.9245283018867925.
[I 2022-12-28 14:24:42,815] Trial 6 finished with value: 0.8301886792452831 and parameters: {'lambda_l1': 0.0189968136862103, 'lambda_l2': 3.483607722774873e-08, 'num_leaves': 172, 'feature_fraction': 0.8023827217708956, 'bagging_fraction': 0.5262295366443046, 'bagging_freq': 1, 'min_child_sa

[LightGBM] [Warning] feature_fraction is set=0.9662488471087745, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9662488471087745
[LightGBM] [Warning] lambda_l1 is set=0.0036200908827975302, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0036200908827975302
[LightGBM] [Warning] bagging_fraction is set=0.80909217946209, subsample=1.0 will be ignored. Current value: bagging_fraction=0.80909217946209
[LightGBM] [Warning] lambda_l2 is set=0.003227464726605418, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.003227464726605418
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.8023827217708956, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8023827217708956
[LightGBM] [Warning] lambda_l1 is set=0.0189968136862103, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0189968136862103
[LightGBM] [Warning] bagging_fr

[I 2022-12-28 14:24:43,145] Trial 10 finished with value: 0.9433962264150944 and parameters: {'lambda_l1': 1.0385054537707835e-05, 'lambda_l2': 2.7149358695740567e-05, 'num_leaves': 29, 'feature_fraction': 0.6701590065066593, 'bagging_fraction': 0.9695680863643132, 'bagging_freq': 4, 'min_child_samples': 9}. Best is trial 10 with value: 0.9433962264150944.
[I 2022-12-28 14:24:43,266] Trial 11 finished with value: 0.9433962264150944 and parameters: {'lambda_l1': 1.1099685029721614e-05, 'lambda_l2': 2.2755998518772974e-05, 'num_leaves': 31, 'feature_fraction': 0.628249776699949, 'bagging_fraction': 0.9873922819891425, 'bagging_freq': 4, 'min_child_samples': 6}. Best is trial 10 with value: 0.9433962264150944.
[I 2022-12-28 14:24:43,326] Trial 12 finished with value: 0.8867924528301887 and parameters: {'lambda_l1': 8.682467252700764e-06, 'lambda_l2': 8.784608014740791e-06, 'num_leaves': 8, 'feature_fraction': 0.6072740863582603, 'bagging_fraction': 0.9732455950471068, 'bagging_freq': 5, '

...

[LightGBM] [Warning] feature_fraction is set=0.9870675071742409, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9870675071742409
[LightGBM] [Warning] lambda_l1 is set=0.00022450062404953246, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00022450062404953246
[LightGBM] [Warning] bagging_fraction is set=0.9178226086123319, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9178226086123319
[LightGBM] [Warning] lambda_l2 is set=2.571782802254792e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.571782802254792e-05
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2022-12-28 14:30:50,736] Trial 1997 finished with value: 0.9433962264150944 and parameters: {'lambda_l1': 0.00022450062404953246, 'lambda_l2': 2.571782802254792e-05, 'num_leaves': 10, 'feature_fraction': 0.9870675071742409, 'bagging_fraction': 0.9178226086123319, 'bagging_freq': 1, 'min_child_samples': 7}. Best is trial 933 with value: 1.0.
[I 2022-12-28 14:30:51,003] Trial 1998 finished with value: 0.9433962264150944 and parameters: {'lambda_l1': 5.601531761922169e-07, 'lambda_l2': 1.9512682091134973e-05, 'num_leaves': 43, 'feature_fraction': 0.9744268540750706, 'bagging_fraction': 0.9667542620913747, 'bagging_freq': 3, 'min_child_samples': 11}. Best is trial 933 with value: 1.0.


[LightGBM] [Warning] feature_fraction is set=0.9744268540750706, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9744268540750706
[LightGBM] [Warning] lambda_l1 is set=5.601531761922169e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.601531761922169e-07
[LightGBM] [Warning] bagging_fraction is set=0.9667542620913747, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9667542620913747
[LightGBM] [Warning] lambda_l2 is set=1.9512682091134973e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.9512682091134973e-05
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


[I 2022-12-28 14:30:51,167] Trial 1999 finished with value: 0.8113207547169812 and parameters: {'lambda_l1': 3.807070038941763e-08, 'lambda_l2': 4.941568059944079e-08, 'num_leaves': 115, 'feature_fraction': 0.5661960813179052, 'bagging_fraction': 0.8822874471718791, 'bagging_freq': 5, 'min_child_samples': 49}. Best is trial 933 with value: 1.0.


[LightGBM] [Warning] feature_fraction is set=0.5661960813179052, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5661960813179052
[LightGBM] [Warning] lambda_l1 is set=3.807070038941763e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.807070038941763e-08
[LightGBM] [Warning] bagging_fraction is set=0.8822874471718791, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8822874471718791
[LightGBM] [Warning] lambda_l2 is set=4.941568059944079e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.941568059944079e-08
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


In [27]:
study.best_params

{'lambda_l1': 2.722250026132842e-05,
 'lambda_l2': 1.577574885335236e-05,
 'num_leaves': 111,
 'feature_fraction': 0.821344005950656,
 'bagging_fraction': 0.8217265699689372,
 'bagging_freq': 7,
 'min_child_samples': 5}

In [28]:
# Best Parameter로 모델 훈련
clfs = []
clf = LGBMClassifier(**study.best_params)
clf.fit(x_train, y_train)
clfs.append(clf)


# 예측 수행 (soft voting)
submit = pd.read_csv('./data/sample_submission.csv')
pred = [clf.predict_proba(test) for clf in clfs]
pred = np.mean(pred, axis=0)
submit['class'] = TARLE.inverse_transform(np.argmax(pred, axis=1))

[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] lambda_l1 is set=2.722250026132842e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.722250026132842e-05
[LightGBM] [Warning] feature_fraction is set=0.821344005950656, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.821344005950656
[LightGBM] [Warning] bagging_fraction is set=0.8217265699689372, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8217265699689372
[LightGBM] [Warning] lambda_l2 is set=1.577574885335236e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.577574885335236e-05


In [31]:
submit_9906 = pd.read_csv('./trial 14 (99.06%).csv')
check = []
for idx in range(len(submit)):
    if submit['class'][idx] == submit_9906['class'][idx]: check.append(True)
    else: check.append(False)
pd.DataFrame(check).value_counts()

True     169
False      6
dtype: int64

In [30]:
# submit.to_csv('./trial 17.csv', index=False)